In [1]:
import findspark
import happybase
import pandas as pd
import numpy as np

In [2]:
findspark.init()

In [40]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
spark = (
    SparkSession.builder
    .appName("StackExchange")
    .getOrCreate()
)

In [41]:
VM_address = 'localhost'
conn = happybase.Connection(VM_address)

In [22]:
table_name = 'stackoverflow'
table_stackoverflow = conn.table(table_name)

In [23]:
for key, data in table_stackoverflow.scan(limit=3):
    print(f'KEY: {key}')
    for column, value in data.items():
        print(f'\tCOLUMN: {column} | VALUE: {value}')


KEY: b'75430619'
	COLUMN: b'owner:account_id' | VALUE: b'117126'
	COLUMN: b'owner:reputation' | VALUE: b'545'
	COLUMN: b'owner:user_id' | VALUE: b'306857'
	COLUMN: b'owner:user_type' | VALUE: b'registered'
	COLUMN: b'post_details:answer_count' | VALUE: b'1'
	COLUMN: b'post_details:creation_date' | VALUE: b'1676241304'
	COLUMN: b'post_details:is_answered' | VALUE: b'true'
	COLUMN: b'post_details:last_activity_date' | VALUE: b'1676246557'
	COLUMN: b'post_details:score' | VALUE: b'0'
	COLUMN: b'post_details:title' | VALUE: b'Echo input into output, at the right place'
	COLUMN: b'post_details:view_count' | VALUE: b'106'
	COLUMN: b'tags:tags' | VALUE: b'["c","input","io","user-input","io-redirection"]'
KEY: b'75430621'
	COLUMN: b'owner:account_id' | VALUE: b'24174919'
	COLUMN: b'owner:reputation' | VALUE: b'13'
	COLUMN: b'owner:user_id' | VALUE: b'18136532'
	COLUMN: b'owner:user_type' | VALUE: b'registered'
	COLUMN: b'post_details:answer_count' | VALUE: b'0'
	COLUMN: b'post_details:creation

### Wczytanie danych do dataFrame

In [24]:
df_online_rows=[]
df_online_tags_rows=[]
for key, data in table_stackoverflow.scan():
    last_edit_date = int(data.get(b'post_details:last_edit_date', b'0').decode('UTF-8'))
    if last_edit_date == 0:
        last_edit_date = None
        
    owner_account_id = int(data.get(b'owner:account_id', b'0').decode('UTF-8'))
    if owner_account_id == 0:
        owner_account_id = None
        
    owner_reputation = int(data.get(b'owner:reputation', b'-1000000000').decode('UTF-8'))
    if owner_reputation == -1000000000:
        owner_reputation = None
    
    owner_user_id = int(data.get(b'owner:user_id', b'-1000000000').decode('UTF-8'))
    if owner_user_id == -1000000000:
        owner_user_id = None
    new_row = {
        'owner_account_id': owner_account_id,
        'owner_reputation': owner_reputation,
        'owner_user_id': owner_user_id,
        'owner_user_type': data[b'owner:user_type'].decode('UTF-8'),
        'post_answer_count': int(data[b'post_details:answer_count'].decode('UTF-8')),
        'post_creation_date': int(data[b'post_details:creation_date'].decode('UTF-8')),
        'post_is_answered': bool(data[b'post_details:is_answered'].decode('UTF-8')),
        'post_last_activity_date': int(data[b'post_details:last_activity_date'].decode('UTF-8')),
        'post_last_edit_date': last_edit_date,
        'post_score': int(data[b'post_details:score'].decode('UTF-8')),
        'post_title': data[b'post_details:title'].decode('UTF-8'),
        'post_view_count': int(data[b'post_details:view_count'].decode('UTF-8')),
        'question_id': int(key.decode('UTF-8'))
    }
    df_online_rows.append(new_row)
    tags = [t.strip('"') for t in data[b'tags:tags'].decode('UTF-8').strip('[]').split(',')]
    for tag in tags:
        df_online_tags_rows.append({'question_id': new_row['question_id'],
                                   'tag': tag})
df_online = pd.DataFrame(df_online_rows)
df_online_tags=pd.DataFrame(df_online_tags_rows)

In [29]:
df_online_tags['tag'].value_counts()

tag
python              4886
javascript          3226
reactjs             1932
java                1705
c#                  1571
                    ... 
telnetlib3             1
onvif                  1
honeywell              1
mybatis-plus           1
windows-firewall       1
Name: count, Length: 13636, dtype: int64

In [42]:
table_name = 'datascience_posts'
table_datascience_posts = conn.table(table_name)
table_name = 'datascience_users'
table_datascience_users = conn.table(table_name)

In [36]:
for key, data in table_datascience_posts.scan(limit=3):
    print(f'KEY: {key}')
    for column, value in data.items():
        print(f'\tCOLUMN: {column} | VALUE: {value}')


KEY: b'10000'
	COLUMN: b'post:answerCount' | VALUE: b'1'
	COLUMN: b'post:commentCount' | VALUE: b'2'
	COLUMN: b'post:creationDate' | VALUE: b'2016-01-27T19:58:00.083'
	COLUMN: b'post:lastActivityDate' | VALUE: b'2017-12-27T20:43:21.200'
	COLUMN: b'post:lastEditDate' | VALUE: b'2016-01-27T21:00:53.283'
	COLUMN: b'post:ownerUserId' | VALUE: b'8820'
	COLUMN: b'post:score' | VALUE: b'15'
	COLUMN: b'post:viewCount' | VALUE: b'7402'
	COLUMN: b'tags:tags' | VALUE: b'<bayesian-networks><pgm>'
KEY: b'100001'
	COLUMN: b'post:answerCount' | VALUE: b'2'
	COLUMN: b'post:commentCount' | VALUE: b'0'
	COLUMN: b'post:creationDate' | VALUE: b'2021-08-12T18:39:11.853'
	COLUMN: b'post:lastActivityDate' | VALUE: b'2021-08-18T12:36:43.523'
	COLUMN: b'post:ownerUserId' | VALUE: b'121019'
	COLUMN: b'post:score' | VALUE: b'1'
	COLUMN: b'post:viewCount' | VALUE: b'215'
	COLUMN: b'tags:tags' | VALUE: b'<machine-learning><nlp><transformer><machine-translation>'
KEY: b'100003'
	COLUMN: b'post:answerCount' | VALUE:

In [43]:
df_static_posts_rows=[]
for key, data in table_datascience_posts.scan():
    last_edit_date = data.get(b'post:lastEditDate', b'').decode('UTF-8')
    if last_edit_date == '':
        last_edit_date = None
    
    closed_date = data.get(b'post:closedDate', b'').decode('UTF-8')
    if closed_date == '':
        closed_date = None
        
    new_row = {
        'answerCount': int(data[b'post:answerCount'].decode('UTF-8')),
        'commentCount': int(data[b'post:commentCount'].decode('UTF-8')),
        'creationDate': data[b'post:creationDate'].decode('UTF-8'),
        'lastActivityDate': data[b'post:lastActivityDate'].decode('UTF-8'),
        'lastEditDate': last_edit_date,
        'closedDate': closed_date,
        'ownerUserId': int(data[b'post:ownerUserId'].decode('UTF-8')),
        'score': int(data[b'post:score'].decode('UTF-8')),
        'viewCount': int(data[b'post:viewCount'].decode('UTF-8')),
        'tags': data[b'tags:tags'].decode('UTF-8'),
        'question_id': key.decode('UTF-8'),
    }
    df_static_posts_rows.append(new_row)
#     tags = [t.strip('"') for t in data[b'tags:tags'].decode('UTF-8').strip('[]').split(',')]
#     for tag in tags:
#         df_online_tags_rows.append({'question_id': new_row['question_id'],
#                                    'tag': tag})
df_static_posts = pd.DataFrame(df_static_posts_rows)
#df_online_tags=pd.DataFrame(df_online_tags_rows)

KeyError: b'post:ownerUserId'

In [37]:
for key, data in table_datascience_users.scan(limit=3):
    print(f'KEY: {key}')
    for column, value in data.items():
        print(f'\tCOLUMN: {column} | VALUE: {value}')


KEY: b'-1'
	COLUMN: b'user:creationDate' | VALUE: b'2014-05-13T21:29:22.820'
	COLUMN: b'user:downVotes' | VALUE: b'2148'
	COLUMN: b'user:reputation' | VALUE: b'1'
	COLUMN: b'user:upVotes' | VALUE: b'841'
	COLUMN: b'user:views' | VALUE: b'495'
KEY: b'1'
	COLUMN: b'user:creationDate' | VALUE: b'2014-05-13T22:58:54.810'
	COLUMN: b'user:downVotes' | VALUE: b'0'
	COLUMN: b'user:reputation' | VALUE: b'101'
	COLUMN: b'user:upVotes' | VALUE: b'0'
	COLUMN: b'user:views' | VALUE: b'723'
KEY: b'10'
	COLUMN: b'user:creationDate' | VALUE: b'2014-05-13T23:18:31.953'
	COLUMN: b'user:downVotes' | VALUE: b'0'
	COLUMN: b'user:reputation' | VALUE: b'101'
	COLUMN: b'user:upVotes' | VALUE: b'1'
	COLUMN: b'user:views' | VALUE: b'5'


In [ ]:
df_static_users_rows=[]
for key, data in table_datascience_posts.scan():
    last_edit_date = int(data.get(b'post_details:last_edit_date', b'0').decode('UTF-8'))
    if last_edit_date == 0:
        last_edit_date = None
        
    owner_account_id = int(data.get(b'owner:account_id', b'0').decode('UTF-8'))
    if owner_account_id == 0:
        owner_account_id = None
        
    owner_reputation = int(data.get(b'owner:reputation', b'-1000000000').decode('UTF-8'))
    if owner_reputation == -1000000000:
        owner_reputation = None
    
    owner_user_id = int(data.get(b'owner:user_id', b'-1000000000').decode('UTF-8'))
    if owner_user_id == -1000000000:
        owner_user_id = None
    new_row = {
        'owner_account_id': owner_account_id,
        'owner_reputation': owner_reputation,
        'owner_user_id': owner_user_id,
        'owner_user_type': data[b'owner:user_type'].decode('UTF-8'),
        'post_answer_count': int(data[b'post_details:answer_count'].decode('UTF-8')),
        'post_creation_date': int(data[b'post_details:creation_date'].decode('UTF-8')),
        'post_is_answered': bool(data[b'post_details:is_answered'].decode('UTF-8')),
        'post_last_activity_date': int(data[b'post_details:last_activity_date'].decode('UTF-8')),
        'post_last_edit_date': last_edit_date,
        'post_score': int(data[b'post_details:score'].decode('UTF-8')),
        'post_title': data[b'post_details:title'].decode('UTF-8'),
        'post_view_count': int(data[b'post_details:view_count'].decode('UTF-8')),
        'question_id': int(key.decode('UTF-8'))
    }
    df_online_rows.append(new_row)
    tags = [t.strip('"') for t in data[b'tags:tags'].decode('UTF-8').strip('[]').split(',')]
    for tag in tags:
        df_online_tags_rows.append({'question_id': new_row['question_id'],
                                   'tag': tag})
df_online = pd.DataFrame(df_online_rows)
df_online_tags=pd.DataFrame(df_online_tags_rows)